In [1]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg

In [2]:
spark = (SparkSession.builder
        .appName("Ages")
        .getOrCreate())

23/01/31 02:01:16 WARN Utils: Your hostname, Deboras-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.68.104 instead (on interface en0)
23/01/31 02:01:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/31 02:01:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Using RDD

In [3]:
#Agregate all ages by name and get the average name by age
#Create an RDD of tuples (name, age)

dataRDD = spark.sparkContext.parallelize([("Pedro", 38), ("Maria", 42), ("João", 12), ("Maria", 10)])

avgRDD = (dataRDD \
    .map(lambda x : (x[0], (x[1],1))) \
    .reduceByKey(lambda x,y : (x[0] + y[0], x[1] + y[1])) \
    .map(lambda x : (x[0], x[1][0]/x[1][1]))
    )

avgRDD.collect()

[('João', 12.0), ('Pedro', 38.0), ('Maria', 26.0)]

### Using Dartaframe

In [4]:
data_df = spark.createDataFrame([("Pedro", 38), ("Maria", 42), ("João", 12), ("Maria", 10)], ["nome", "idade"])

avg_df = data_df.groupBy("nome").agg(avg("idade"))

avg_df.show()

+-----+----------+
| nome|avg(idade)|
+-----+----------+
|Pedro|      38.0|
|Maria|      26.0|
| João|      12.0|
+-----+----------+



In [5]:
young = data_df.filter(data_df.idade < 18)
young.show()

+-----+-----+
| nome|idade|
+-----+-----+
| João|   12|
|Maria|   10|
+-----+-----+



In [6]:
old = data_df[data_df.idade > 18]
old.show()

+-----+-----+
| nome|idade|
+-----+-----+
|Pedro|   38|
|Maria|   42|
+-----+-----+



In [7]:
young.select(young.nome, young.idade + 1).show()

+-----+-----------+
| nome|(idade + 1)|
+-----+-----------+
| João|         13|
|Maria|         11|
+-----+-----------+



In [8]:
spark.stop()